In [ ]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import os

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [ ]:
# Obtiene el promedio espacual por municipio.
def proc_mean(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_yearly.nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        if k == 0:
            df_y = df[ df["Año"] == y ].copy()
        else: df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat"])
                .to_dataframe()
                .drop(columns = "CVE_INEGI")
                .mean()[0] )
        return df_y

# Obtiene el promedio temporal y opera con los valores mensuales
def proc_mean_month(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_monthly.nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        if k == 0:
            df_y = df[ df["Año"] == y ].copy()
        else: df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_m = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe()
                .drop(columns = "CVE_INEGI")
                .iloc[:, 0] )
            # Ordenamos los datos de mayor a menor.
            df_m_1 = df_m.sort_values(ascending = False)
            # Ordenamos la suma de 6 meses
            # (verano) de mayor a menor
            df_m_2 = ( df_m.loc[::-1]
              .rolling(6).mean()
              .sort_values(ascending = False) )
            # Segundo mes más alto.
            df_y.loc[ df["CVE_INEGI"] == i,
                "Tmean_max_2" ] = df_m_1.iloc[1]
            # Mes de inicio de la temporada más calida.
            df_y.loc[ df["CVE_INEGI"] == i,
                "M_verano" ] = df_m_2.index[0]
            # Temperatura media.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = df_m.mean()
        return df_y
  
# Obtiene la suma temporal y espacial.
def proc_soc(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_yearly.nc" ) as ds:
        df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            #  de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .sum("lat").to_dataframe()
                .drop(columns = "CVE_INEGI")
                .sum()[0] )
        return df_y

In [ ]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 

# Escenarios y años.
RCP = ["RCP4p5", "RCP8p5"]
years = list(range(2010, 2017))

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/" for x in dirs]
folder = "vars/"
path_d = [path + folder + x + "/" for x in dirs]


# Variables para generar la máscara.
vars = ["Tmax", "Tmin", "Tmean", "Pre",
        "HDD_mean", "CDD_mean", "HDD_p10",
        "CDD_p90", "Pre_Tmean", "Poblacion", "PIB"]

# Nombre de los archivos.
f_mask = "/Municipios/municipios_mask.nc"
csv = ".csv"

# Máscara para los municipios.
mask_0 = xr.open_dataset( path + f_mask )

In [ ]:
# Procesa los datos actuales.
# Procesado manual para cada variable porque
# es más tardado que el procesado futuro.

# Variable a procesar, 0 - 8
n = 8
var = vars[n]

mask = mask_0.copy()

if var in vars[4:8]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var)

# Datos actuales 
k = 0

# Abrimos los archivos a utilizar.
df = pd.read_csv( path_data[k]
    + "data_0" + csv)
df_r = []

# Procesamos los años en paralelo.
pool = Pool()
if var == "Tmean":
    df_r = pool.map(proc_mean_month, years)
else:
    # Variables que se promedian.
    df_r = pool.map(proc_mean, years)

# Concatenamos los resultados de cada año.
for p in df_r:
    df.loc[df["Año"] == p["Año"].max() ] = p

# Guardamos el resultado como un nuevo archivo.
df.to_csv(path_data[k] + "data_" +
    str(n + 1) + csv, index = False)

In [ ]:
df_r[0].head()

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Consumo_DAC,Usuarios_DAC,Tarifa,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Año
0,1001,1,1,Aguascalientes,Aguascalientes,NaN,NaN,NaN,NaN,NaN,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,NaN,NaN,24.024788,5.0,NaN,NaN,NaN,NaN,116635.924759,2010
1,1002,1,2,Aguascalientes,Asientos,NaN,NaN,NaN,NaN,NaN,-102.045590,22.126479,26.162073,11.428409,18.791330,345.914682,-693.016602,542.496094,-24.048777,NaN,295.488361,NaN,NaN,23.582296,5.0,NaN,NaN,NaN,NaN,54337.426603,2010
2,1003,1,3,Aguascalientes,Calvillo,NaN,NaN,NaN,NaN,NaN,-102.704911,21.900640,26.601974,11.342110,18.997988,514.103828,-674.639343,405.724884,-12.293874,NaN,432.323312,NaN,NaN,23.260357,5.0,NaN,NaN,NaN,NaN,92335.783229,2010
3,1004,1,4,Aguascalientes,CosÃ­o,NaN,NaN,NaN,NaN,NaN,-102.297038,22.360619,26.273094,11.475876,18.911726,375.646958,-715.177002,544.481018,-28.443644,NaN,320.155330,NaN,NaN,23.338507,5.0,NaN,NaN,NaN,NaN,12838.354454,2010
4,1005,1,5,Aguascalientes,JesÃºs MarÃ­a,NaN,NaN,NaN,NaN,NaN,-102.445700,21.932112,26.563141,11.753613,19.136818,432.641368,-685.458801,439.140594,-16.992479,0.022585,369.465312,NaN,NaN,23.525841,5.0,NaN,NaN,NaN,NaN,49989.509158,2010


In [ ]:
# Procesa los datos futuros.

fut = [2030, 2050, 2070]

for n in range(11):

    # Variable a procesar, con e = 0, 0 - 8; con e = 1, 0 - 10.
    var = vars[n]

    mask = mask_0.copy()

    if var in vars[4:8]:
        mask = mask.rename_vars(mask = "degree_day")
    else:
        mask = mask.rename_vars(mask = var)
    
    # Para el PIB y la población,
    # se agrega un año actual.
    if n in [9, 10]:
        fut = [2010] + fut
    
    # Iteramos en los RCPs.
    for k in range(len(path_d[1:])):
        k += 1
        
        # Iteramos en los años.
        for f in fut:
            b = 1

            # Abrimos los archivos a utilizar.
            df = pd.read_csv( path_data[k] +
                "data_0." + str(f) + csv )
            df_r = []

            # Población  PIB
            if var in vars[9:11]:
                # Por el momento, solo para RCP8.5
                if k != 1:
                    fut = [2010] + fut
                    df_r = proc_soc(f)
                    # Elimina el respaldo, si existe.
                    if os.path.exists( path_data[k] +
                        "data_" + str(n + 1) + "." +
                        str(f) + ".original.csv" ):
                        os.remove( path_data[k] +
                        "data_" + str(n + 1) + "." +
                        str(f) + ".original.csv" )
                else: b = 0
            # Calcula además la temperatura mínima
            # máxima y el mes de inicio del verano.
            elif var == "Tmean":
                df_r = proc_mean_month(f)
            # Variables que se promedian.
            else:
                df_r = proc_mean(f)

            if b:
                # Guardamos el resultado como un nuevo archivo.
                df_r.to_csv(path_data[k] + "data_" + str(n + 1)
                    + "." + str(f) + ".csv", index = False)